## Pré: Conexão com Google Drive

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Etapa 1: Instalação das dependências

In [23]:
!pip install tensorflow-gpu==2.1.0
!pip install tqdm
!pip install kaggle
!pip install keras-rectified-adam
!pip install tensorflow-addons
#!pip install tfa-nightly

     |████████████████████████████████| 1.0MB 4.9MB/s 


### Importação das bibliotecas

In [24]:
import zipfile
import os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import pandas as pd
import sys
import itertools
from keras_radam import RAdam
import shutil

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.1.0'

### Otimizações tensorflow

In [0]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)
tf.config.optimizer.set_jit(True)

### Fazendo o download da base de dados de imagens

In [26]:
%%time
if (os.path.isdir('./ISIC_2019_Test_Input')==False):
  !wget --no-check-certificate \
      https://s3.amazonaws.com/isic-challenge-2019/ISIC_2019_Test_Input.zip \
      -O ./skin_cancer_test.zip

CPU times: user 30 µs, sys: 4 µs, total: 34 µs
Wall time: 39.3 µs


## Etapa 2: Pré-processamento

### Configurando os caminhos (paths)

In [0]:
test_dir = "./test"

download_path = './'
derm_ai_path_V1 = '/content/drive/My Drive/Colab Notebooks/Dermatologist_AI'
derm_ai_path = '/content/drive/My Drive/Colab Notebooks/Dermatologist_AI/ISIC_2019'
#ground_truth_file = os.path.join(derm_ai_path, 'ground_truth.csv')

In [0]:
def dcp(download_path, path, file):
  if (os.path.isdir(path)==False):
    dataset_path = os.path.join(download_path,file)
    zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
    zip_object.extractall("./")
    zip_object.close()

### Descompactando a base de dados de Treinamento

### Descompactando a base de dados de Testes

In [29]:
%%time
dcp(download_path, './ISIC_2019_Test_Input', 'skin_cancer_test.zip')

CPU times: user 57 µs, sys: 7 µs, total: 64 µs
Wall time: 48.9 µs


In [30]:
!rm ./skin_cancer_test.zip

rm: cannot remove './skin_cancer_test.zip': No such file or directory


### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: InceptionResNetV2 (que estamos usando) suporta: (299x299).

In [0]:
data_gen_test = ImageDataGenerator(rescale=1/255., 
                                   fill_mode="constant"
)

In [32]:
!mkdir ./test
!mkdir ./test/test
!mv ./ISIC_2019_Test_Input/* ./test/test

mkdir: cannot create directory ‘./test’: File exists
mkdir: cannot create directory ‘./test/test’: File exists
mv: cannot stat './ISIC_2019_Test_Input/*': No such file or directory


In [33]:
test_generator  = data_gen_test.flow_from_directory(test_dir, target_size=(299,299), batch_size=32, shuffle=False, class_mode=None)

Found 8238 images belonging to 1 classes.


#### Exibição de 3 imagens carregadas

In [36]:
x,y = test_generator.next()
for i in range(0,3):
    image = x[i]
    label = y[i]
    print (label)
    plt.imshow(image)
    plt.show()

ValueError: ignored

## Etapa 3: Construindo o modelo

### Carregando o modelo pré-treinado (InceptionResNetV2)

### Congelando o modelo base

### Definindo o cabeçalho personalizado da rede neural

### Definindo o modelo

### Compilando o modelo

### Treinando o modelo

### Avaliação do modelo de transferência de aprendizagem

## Etapa 4: Fine tuning


Algumas questões principais:

- NEM SEMPRE é necessário realizar o Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 
- Mais informações em como usar o Fine Tuning acesse: 
 https://www.analyticsvidhya.com/blog/2017/06/transfer-learning-the-art-of-fine-tuning-a-pre-trained-model/

### Descongelando algumas camadas do topo do modelo base

Descomentar trecho abaixo para manter camadas congeladas. O número informado no parâmetro `fine_tuning_at` define o número das primeiras camadas que ficarão congeladas. Supondo que a rede tenha 780 camadas e seja informado o valor 500, até a camada 500 a rede ficará congelada.

### Compilando o modelo para fine tuning

### Fine tuning

### Avaliação do modelo com fine tuning

### Salvar modelo após Fine Tuning

In [39]:
%%time
img_shape = (299, 299, 3)
base_model = tf.keras.applications.InceptionResNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

#base_model.output
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
#global_average_layer
prediction_layer = tf.keras.layers.Dense(units = 8, activation = "softmax")(global_average_layer)

model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

radam = tfa.optimizers.RectifiedAdam(
    lr=1e-2,
    total_steps=634,
    warmup_proportion=0.1,
    min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)

optm = ranger

model.compile(optimizer=optm, loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy()])

CPU times: user 10.9 s, sys: 399 ms, total: 11.3 s
Wall time: 10.9 s


### Carregar modelo após Fine Tuning com os melhores pesos

In [40]:
%%time

InceptionResNetV2_model_file = os.path.join(derm_ai_path, 'InceptionResNetV2_full_cat.h5')
checkpoint_filepath = os.path.join(derm_ai_path, 'InceptionResNetV2_cat.weights.best.my.hdf5')

# Recreate the exact same model, including its weights and the optimizer
#model = tf.keras.models.load_model(InceptionResNetV2_model_file)
model.load_weights(checkpoint_filepath)


CPU times: user 1.02 s, sys: 185 ms, total: 1.2 s
Wall time: 3.05 s


### Matriz de Validação

### Execução das predições na base de testes

In [41]:
%%time

test_generator.reset()
pred=model.predict_generator(test_generator,
                              verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
258/258 [==============================] - 182s 707ms/step
CPU times: user 3min 19s, sys: 19.2 s, total: 3min 38s
Wall time: 3min 2s


### Identificar classe predita

In [0]:
#def predClasses(pred, train_generator):
#  predicted_class_indices=np.argmax(pred,axis=1)
#
#  labels = (train_generator.class_indices)
#  labels = dict((v,k) for k,v in labels.items())
#  predictions = [labels[k] for k in predicted_class_indices]
#  return predicted_class_indices, labels, predictions

#predicted_class_indices, labels, predictions = predClasses(pred, train_generator)

In [0]:
#image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
#labels[5]
#predicted_class_indices[0]
#predictions[0]
#predicted_class_indices[0]
#pred[0].max()

In [0]:
def is_unknown(pred, th):
  unk = []
  for row in pred:
    if row.max() < th:
        #Unknown class
        unk.append(1.0)
    else:
        unk.append(0.0)
  return unk

th = 0.35

### Salvar resultado para conferência

In [0]:
pd.set_option('display.float_format', lambda x: '%.12f' % x)
results_file = os.path.join(derm_ai_path, 'results_InceptionResNetV2_full_cat.csv')

def saveResultsFile(results_file,pred, test_generator, unk):
  filenames=test_generator.filenames
  filenames = [w.replace('test/', '') for w in filenames]
  filenames = [w.replace('.jpg', '') for w in filenames]
  results=pd.DataFrame({"image":filenames,
                        "MEL":pred[:,4],
                        "NV":pred[:,5],
                        "BCC":pred[:,1],
                        "AK":pred[:,0],
                        "BKL":pred[:,2],
                        "DF":pred[:,3],
                        "VASC":pred[:,7],
                        "SCC":pred[:,6],
                        "UNK": unk
                        })
  results.to_csv(results_file,index=False, float_format='%.12f')

unk = is_unknown(pred, th)
saveResultsFile(results_file, pred, test_generator, unk)

In [45]:
df_comp = pd.read_csv(results_file)
df_comp.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0034321,0.192059680820,0.202574446797,0.035144556314,0.000082273524,0.453795969486,0.088554814458,0.022022530437,0.005765677430,0.000000000000
1,ISIC_0034322,0.000260127010,0.999150276184,0.000017533983,0.000013911616,0.000075475858,0.000256122206,0.000210870741,0.000015777792,0.000000000000
2,ISIC_0034323,0.000000353523,0.000000021792,0.998624324799,0.000338717655,0.000001996758,0.000364080304,0.000001694103,0.000668972090,0.000000000000
3,ISIC_0034324,0.001458322397,0.996295630932,0.000007099342,0.000002314083,0.001927197212,0.000182376811,0.000118680291,0.000008414805,0.000000000000
4,ISIC_0034325,0.002522030612,0.996125936508,0.000015358648,0.000024743445,0.000974733091,0.000220531496,0.000075963158,0.000040660500,0.000000000000


### Rodar avaliação do modelo em relação a base de testes

### Salvar modelo com score obtido

### Carregar melhor modelo 

## Etapa 5: Tensorflow Lite

### Converter modelo para formato tflite

### Salvar arquivo convertido

### Testar modelo convertido (TFLite)
https://medium.com/towards-artificial-intelligence/testing-tensorflow-lite-image-classification-model-e9c0100d8de3

https://colab.research.google.com/github/frogermcs/TFLite-Tester/blob/master/notebooks/Testing_TFLite_model.ipynb#scrollTo=6WEyRJNsR5uL

https://aihub.cloud.google.com/p/products%2F6ca92c9c-6932-45f8-8f08-15a1a6dfafc7